# 🔍 XMPP Client Upload Authentication Diagnosis

**Problem Analysis:** Dino and Gajim can't upload after restart, Android works after reconnection

**Network Setup:**
- Desktop: WLAN + Ethernet → Router → HMAC File Server
- Mobile: Android XMPP client → Router → HMAC File Server

**Date:** August 26, 2025

## 🎯 Problem Identification

### Symptoms:
- ❌ **Dino (Desktop):** Upload fails after restart
- ❌ **Gajim (Desktop):** Upload fails after restart  
- ✅ **Android:** Upload works after disconnect/reconnect

### Network Context:
- Notebook with WLAN + Ethernet (dual interface)
- Router provides access to HMAC File Server
- Fixed connections vs mobile reconnection behavior

In [ ]:
# Check current server status and configuration
import subprocess
import json
from datetime import datetime

print("🔍 HMAC File Server Status Check")
print("=" * 40)

# Check if server is running
try:
    result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
    if 'hmac-file-server' in result.stdout:
        print("✅ HMAC File Server is running")
        
        # Extract server process info
        for line in result.stdout.split('\n'):
            if 'hmac-file-server' in line and 'grep' not in line:
                print(f"📊 Process: {line.split()[1]} {' '.join(line.split()[10:])}")
    else:
        print("❌ HMAC File Server not running")
except Exception as e:
    print(f"⚠️  Could not check server status: {e}")

print(f"\n🕐 Check time: {datetime.now()}")

## 🔍 Root Cause Analysis

### Likely Issues:

#### 1. **Token Expiration vs Session Management**
- Desktop clients (Dino/Gajim) may cache expired tokens after restart
- Android reconnection triggers fresh token generation
- Grace periods may not apply to cached tokens

#### 2. **Network Interface Detection**
- Dual interface (WLAN + Ethernet) may confuse IP detection
- Desktop clients may use different IP after restart
- Router NAT may assign different internal IPs

#### 3. **Client Behavior Differences**
- Desktop clients: Restore session from disk cache
- Mobile clients: Fresh authentication after reconnect
- Token validation may be stricter for cached sessions

In [ ]:
# Check network configuration and IP detection
print("🌐 Network Configuration Analysis")
print("=" * 40)

# Check network interfaces
try:
    result = subprocess.run(['ip', 'addr', 'show'], capture_output=True, text=True)
    interfaces = []
    current_interface = None
    
    for line in result.stdout.split('\n'):
        if ': ' in line and ('wlan' in line or 'eth' in line or 'eno' in line or 'wlp' in line):
            current_interface = line.split(':')[1].strip().split('@')[0]
            interfaces.append(current_interface)
        elif current_interface and 'inet ' in line and '127.0.0.1' not in line:
            ip = line.strip().split()[1].split('/')[0]
            print(f"📡 Interface {current_interface}: {ip}")
            
    print(f"\n🔌 Total network interfaces found: {len(interfaces)}")
    if len(interfaces) > 1:
        print("⚠️  Multiple interfaces detected - potential IP confusion for clients")
        
except Exception as e:
    print(f"⚠️  Could not analyze network interfaces: {e}")

# Check routing table
try:
    result = subprocess.run(['ip', 'route', 'show'], capture_output=True, text=True)
    print("\n🛣️  Default routes:")
    for line in result.stdout.split('\n'):
        if 'default' in line:
            print(f"   {line}")
except Exception as e:
    print(f"⚠️  Could not check routing: {e}")

## 📊 Bearer Token Analysis

Let's examine how the HMAC File Server handles different client scenarios:

In [ ]:
# Analyze Bearer token validation logic
print("🔐 Bearer Token Validation Analysis")
print("=" * 40)

# Check if the enhanced validation function exists
try:
    with open('/root/hmac-file-server/cmd/server/main.go', 'r') as f:
        content = f.read()
        
    # Look for mobile client detection
    if 'isMobileXMPP' in content:
        print("✅ Mobile XMPP client detection enabled")
        
        # Extract mobile detection logic
        lines = content.split('\n')
        in_mobile_section = False
        for i, line in enumerate(lines):
            if 'isMobileXMPP.*:=' in line or 'isMobileXMPP =' in line:
                in_mobile_section = True
                print("\n📱 Mobile client detection logic:")
            elif in_mobile_section and 'conversations' in line.lower():
                print(f"   - Conversations: {'✅' if 'conversations' in line else '❌'}")
            elif in_mobile_section and 'dino' in line.lower():
                print(f"   - Dino: {'✅' if 'dino' in line else '❌'}")
            elif in_mobile_section and 'gajim' in line.lower():
                print(f"   - Gajim: {'✅' if 'gajim' in line else '❌'}")
            elif in_mobile_section and 'android' in line.lower():
                print(f"   - Android: {'✅' if 'android' in line else '❌'}")
            elif in_mobile_section and ('}' in line or 'if ' in line):
                in_mobile_section = False
                
    # Check grace period configuration
    if 'gracePeriod' in content:
        print("\n⏰ Grace period configuration:")
        for line in content.split('\n'):
            if 'gracePeriod.*=' in line and ('28800' in line or '43200' in line or '86400' in line or '259200' in line):
                if '28800' in line:
                    print("   - Base grace: 8 hours (28800s)")
                elif '43200' in line:
                    print("   - Mobile grace: 12 hours (43200s)")
                elif '86400' in line:
                    print("   - Network resilience: 24 hours (86400s)")
                elif '259200' in line:
                    print("   - Ultra grace: 72 hours (259200s)")
                    
except Exception as e:
    print(f"⚠️  Could not analyze Bearer token validation: {e}")

## 🎯 Specific Problem: Desktop vs Mobile Client Behavior

### The Issue:
1. **Desktop clients (Dino/Gajim)** restore sessions from cache after restart
2. **Cached tokens may be expired** or tied to old IP addresses
3. **Mobile clients get fresh tokens** when reconnecting
4. **Grace periods may not apply** to restored cached sessions

In [ ]:
# Check server logs for authentication failures
print("📋 Recent Authentication Activity")
print("=" * 40)

log_files = [
    '/var/log/hmac-file-server-mobile.log',
    '/var/log/hmac-file-server.log',
    '/tmp/server.log'
]

for log_file in log_files:
    try:
        result = subprocess.run(['tail', '-20', log_file], capture_output=True, text=True)
        if result.returncode == 0 and result.stdout.strip():
            print(f"\n📝 Last 20 lines from {log_file}:")
            lines = result.stdout.strip().split('\n')
            for line in lines[-10:]:  # Show last 10 lines
                if any(keyword in line.lower() for keyword in ['error', 'fail', 'invalid', 'expired', 'bearer', 'auth']):
                    print(f"🔍 {line}")
            break
    except:
        continue
        
print("\n💡 Look for patterns like:")
print("   - 'Invalid Bearer token' (expired cached tokens)")
print("   - 'expired beyond grace period' (old sessions)")
print("   - User-Agent differences between clients")

## 🔧 Solution Strategy

### Immediate Fixes:

#### 1. **Clear Client Caches**
- Dino: `~/.local/share/dino/` 
- Gajim: `~/.local/share/gajim/`

#### 2. **Extend Grace Periods for Desktop Clients**
- Treat Dino/Gajim as mobile clients for grace period calculation
- Add specific detection for desktop XMPP clients

#### 3. **Enhanced Session Recovery**
- Implement session recovery for cached tokens
- Allow IP changes for restored sessions

In [ ]:
# Generate client cache clearing commands
print("🧹 Client Cache Clearing Commands")
print("=" * 40)

import os
home_dir = os.path.expanduser('~')

cache_locations = {
    'Dino': [
        f'{home_dir}/.local/share/dino/',
        f'{home_dir}/.cache/dino/',
        f'{home_dir}/.config/dino/'
    ],
    'Gajim': [
        f'{home_dir}/.local/share/gajim/',
        f'{home_dir}/.cache/gajim/',
        f'{home_dir}/.config/gajim/'
    ]
}

print("🔍 Check these locations for cached data:")
for client, locations in cache_locations.items():
    print(f"\n📱 {client}:")
    for location in locations:
        if os.path.exists(location):
            print(f"   ✅ {location} (exists)")
            # List important files
            try:
                for root, dirs, files in os.walk(location):
                    for file in files:
                        if any(keyword in file.lower() for keyword in ['token', 'session', 'cache', 'upload']):
                            print(f"      🔍 {os.path.join(root, file)}")
            except:
                pass
        else:
            print(f"   ❌ {location} (not found)")

print("\n🚨 MANUAL STEPS TO TRY:")
print("1. Close Dino and Gajim completely")
print("2. Clear application caches (backup first!)")
print("3. Restart clients and test upload")
print("4. If still failing, check server logs for specific errors")

## 🛠️ Enhanced Server Configuration

Let's create an enhanced configuration that treats desktop XMPP clients with the same grace as mobile clients:

In [ ]:
# Check current mobile client detection and suggest improvements
print("🔧 Desktop Client Enhancement Strategy")
print("=" * 40)

# Read current configuration
try:
    with open('/root/hmac-file-server/config-mobile-resilient.toml', 'r') as f:
        config = f.read()
    
    print("📄 Current grace period settings:")
    for line in config.split('\n'):
        if 'grace' in line.lower() and '=' in line:
            print(f"   {line.strip()}")
            
    print("\n💡 Recommended enhancement:")
    print("   - Treat Dino and Gajim as 'mobile' clients for grace periods")
    print("   - Add 'desktop_xmpp_grace_period = 24h' for cached session recovery")
    print("   - Enable session_restoration = true for desktop clients")
    
except Exception as e:
    print(f"⚠️  Could not read config: {e}")

# Show the enhanced mobile detection logic needed
print("\n🔍 Enhanced Client Detection Logic Needed:")
print("```go")
print("// Enhanced XMPP client detection (both mobile and desktop)")
print("isXMPPClient := strings.Contains(strings.ToLower(userAgent), \"conversations\") ||")
print("              strings.Contains(strings.ToLower(userAgent), \"dino\") ||")
print("              strings.Contains(strings.ToLower(userAgent), \"gajim\") ||")
print("              strings.Contains(strings.ToLower(userAgent), \"android\") ||")
print("              strings.Contains(strings.ToLower(userAgent), \"xmpp\")")
print("")
print("// Desktop XMPP clients need same grace as mobile for session restoration")
print("if isXMPPClient {")
print("    gracePeriod = int64(86400) // 24 hours for all XMPP clients")
print("}")
print("```")

## 🎯 Immediate Action Plan

### Step 1: Quick Client Fix
1. **Close Dino and Gajim completely**
2. **Clear their caches/sessions** (backup first)
3. **Restart clients** - they should get fresh tokens

### Step 2: Server Enhancement 
1. **Modify mobile client detection** to include desktop XMPP clients
2. **Extend grace periods** for all XMPP clients (not just mobile)
3. **Add session restoration** logic for cached tokens

### Step 3: Network Optimization
1. **Check for IP conflicts** between WLAN/Ethernet
2. **Verify router configuration** for consistent NAT
3. **Monitor upload endpoints** for client-specific issues

In [ ]:
# Generate immediate fix commands
print("⚡ IMMEDIATE FIX COMMANDS")
print("=" * 40)

print("1️⃣ STOP XMPP CLIENTS:")
print("   pkill -f dino")
print("   pkill -f gajim")
print("   # Wait 5 seconds")

print("\n2️⃣ BACKUP AND CLEAR CACHES:")
print("   # Backup first (optional)")
print("   cp -r ~/.local/share/dino ~/.local/share/dino.backup")
print("   cp -r ~/.local/share/gajim ~/.local/share/gajim.backup")
print("   ")
print("   # Clear session caches")
print("   rm -rf ~/.cache/dino/")
print("   rm -rf ~/.cache/gajim/")
print("   ")
print("   # Clear specific upload-related files (if they exist)")
print("   find ~/.local/share/dino -name '*upload*' -delete 2>/dev/null || true")
print("   find ~/.local/share/gajim -name '*upload*' -delete 2>/dev/null || true")

print("\n3️⃣ RESTART CLIENTS:")
print("   # Start Dino")
print("   dino &")
print("   ")
print("   # Start Gajim")
print("   gajim &")

print("\n4️⃣ TEST UPLOAD:")
print("   # Try uploading a small file in both clients")
print("   # Check server logs for any authentication issues")
print("   tail -f /var/log/hmac-file-server-mobile.log")

print("\n🔍 If this doesn't work, the issue is in the server's client detection logic.")
print("The server may not be treating Dino/Gajim with sufficient grace periods.")

## 📋 Diagnosis Summary

### 🎯 **Root Cause**: Session Cache vs Fresh Authentication

- **Desktop clients (Dino/Gajim)**: Restore cached sessions with potentially expired tokens
- **Mobile clients**: Get fresh authentication after reconnection
- **Server**: May not apply sufficient grace periods to cached/restored sessions

### ✅ **Solution Priority**:
1. **Immediate**: Clear client caches to force fresh authentication
2. **Short-term**: Enhance server to treat desktop XMPP clients with mobile-level grace
3. **Long-term**: Implement proper session restoration for all XMPP clients

### 🔧 **Next Steps**:
Execute the immediate fix commands above, then monitor server logs for authentication patterns.